# Pretraining on .py files

In [1]:
import random
from git.repo import Repo
from pathlib import Path
from rich.markdown import Markdown

let's bring some text in

In [11]:
vllm_path = Path("vllm/")
vllm_repo = Repo(vllm_path)

In [9]:
# Get the latest commit
last_commit = weave_repo.commit()

# Print the commit message
print(last_commit)

90979c38f87c17d53a7cd0eb430373ecb0b64b9a


Naively we can bring all MarkDown files

In [12]:
def find_files(directory, extension="*.py"):
    "Find all files of a given `extension` in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob(extension):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

In [13]:
py_files = find_files(vllm_path)

file = random.choice(py_files)

len(py_files), print(file[1][0:500])

# coding=utf-8
# Adapted from
# https://github.com/huggingface/transformers/blob/v4.28.0/src/transformers/models/gpt2/modeling_gpt2.py
# Copyright 2023 The vLLM team.
# Copyright 2023 CTranslate2, and Michael Feil
# Copyright 2018 The OpenAI Team Authors and HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a 


(96, None)

We should stack the file with some metadata, for instance, the path where the files is coming from.

In [14]:
def stuff(file):
    path, content = file
    return f"<<Begin file>>\nPath:\n{path}\n---------\nContent:\n{content}<<End File>>"

In [15]:
print(stuff(file))

<<Begin file>>
Path:
vllm/model_executor/models/gpt_bigcode.py
---------
Content:
# coding=utf-8
# Adapted from
# https://github.com/huggingface/transformers/blob/v4.28.0/src/transformers/models/gpt2/modeling_gpt2.py
# Copyright 2023 The vLLM team.
# Copyright 2023 CTranslate2, and Michael Feil
# Copyright 2018 The OpenAI Team Authors and HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Inference-only GPTBigCo

In [16]:
import json
from tqdm.auto import tqdm

with open('vllm_python.jsonl', 'w') as json_file:
    for file in tqdm(py_files):
        data = stuff(file)
        json.dump({"text":data}, json_file)
        json_file.write('\n')

  0%|          | 0/96 [00:00<?, ?it/s]

In [17]:
import wandb

In [18]:
with wandb.init(project="vllm_llm"):
    at = wandb.Artifact(name="vllm_python", 
                        description="The .py files from the vllm library",
                        type="dataset",
                        metadata={
                            "url": "https://github.com/vllm-project/vllm.git",
                            "commit":last_commit,
                            "remote": weave_repo.remote().url})
    at.add_file("vllm_python.jsonl")
    
    wandb.log_artifact(at)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin
